In [1]:
import sys, os, time
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import top_k_categorical_accuracy

import alexnet

## Load data and Benchmark models

In [2]:
path = 'ILSVRC2012'

In [3]:
y_val = np.load(path+'/y_val.npy')
y_val_one_hot = to_categorical(y_val, 1000)

In [4]:
model = alexnet.AlexNet()

AlexNet created.
Weights loaded.


In [5]:
%%time
for i in range(5):
    x_val = np.load(path+'/'+'val_'+str((i+1)*10000)+'.npy')
    x_val = x_val - x_val.mean()
    x_val = x_val[:, :, :, ::-1]
    pred = model.predict(x_val, verbose=1)
    y_pred = pred if i==0 else np.vstack((y_pred, pred))

10000/10000 [==============================] - 12s 1ms/step
CPU times: user 48.5 s, sys: 1min 14s, total: 2min 2s
Wall time: 13min 49s


In [6]:
def top_k_accuracy(y_true, y_pred, k=1):
    '''
        From: https://github.com/chainer/chainer/issues/606
        Expects both y_true and y_pred to be one-hot encoded.
    '''
    argsorted_y = np.argsort(y_pred)[:,-k:]
    return np.any(argsorted_y.T == y_true.argmax(axis=1), axis=0).mean()

### Result
论文中在测试时每张图片使用5个crop的测试均值作为最终结果

这里每张图片仅使用中间的crop，所以error rate会高一点

#### top-1 error rate
compare to 0.407 in paper

In [7]:
1-top_k_accuracy(y_val_one_hot, y_pred, k=1)

0.47963999999999996

#### top-5 error rate
compare to 0.182 in paper

In [8]:
1-top_k_accuracy(y_val_one_hot, y_pred, k=5)

0.24087999999999998